In [12]:
# Subclassing 모델 사용 : 
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.datasets import mnist
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, ReLU
from keras.callbacks import EarlyStopping
from keras.utils import register_keras_serializable
from keras.metrics import SparseCategoricalAccuracy, Mean # GradientTape 실습 - 주어진 값의 가중 평균을 계산

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() # 제대로 안 섞여있는 것 같다.

# 구조 변경 (차원)
print(x_train.shape) # (60000, 28, 28)
x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0 # 흑백이기 때문에 1을 준다
x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
print(x_train.shape) # (60000, 28, 28, 1)

(60000, 28, 28)
(60000, 28, 28, 1)


In [3]:
# tf.data.Dataset.from_tensor_slices 사용하기 --- 예시 ------------------
x = np.random.sample((5,2))
print(x)

# dset = tf.data.Dataset.from_tensor_slices(x)
# print(dset) # 객체 만들어짐

dset = tf.data.Dataset.from_tensor_slices(x).shuffle(10000).batch(2) # minibatch가 2 이므로 shape(2,2)씩 
print(dset)
for a in dset:
    print(a)
# -----학습에 참여할 때 섞으면 되기 때문에 train만 섞는다 (x_train, y_train)--------------

[[0.35566739 0.3067859 ]
 [0.51236023 0.8043242 ]
 [0.59257984 0.17344464]
 [0.79490196 0.25692738]
 [0.365231   0.92980042]]
<_BatchDataset element_spec=TensorSpec(shape=(None, 2), dtype=tf.float64, name=None)>
tf.Tensor(
[[0.59257984 0.17344464]
 [0.79490196 0.25692738]], shape=(2, 2), dtype=float64)
tf.Tensor(
[[0.51236023 0.8043242 ]
 [0.365231   0.92980042]], shape=(2, 2), dtype=float64)
tf.Tensor([[0.35566739 0.3067859 ]], shape=(1, 2), dtype=float64)


2025-09-19 11:56:52.194562: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-19 11:56:52.194589: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-19 11:56:52.194601: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-19 11:56:52.194619: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-19 11:56:52.194633: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-09-19 11:56:52.228768: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# train data 섞기 - data slicing, GradientTape
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
print(train_ds)
print(test_ds)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [5]:
# 모델 - subclassing(명시적)
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # Conv block1
        self.conv1 = Conv2D(filters=32, kernel_size=[3,3], padding='valid', use_bias=False, activation='relu')
        self.batch1 = BatchNormalization()
        self.pool1 = MaxPool2D((2,2))
        
        # Conv block2
        self.conv2 = Conv2D(64, (3,3), padding='same', use_bias=False, activation='relu')
        self.batch2 = BatchNormalization()
        self.pool2 = MaxPool2D((2,2))
        
        # Conv block3
        self.conv3 = Conv2D(128, (3,3), padding='same', use_bias=False, activation='relu')
        self.batch3 = BatchNormalization()
        self.pool3 = MaxPool2D((2,2))
        
        self.flat = Flatten(dtype='float32')
        
        self.d1 = Dense(32, activation='relu')
        self.do1 = Dropout(0.3)
        self.d2 = Dense(16, activation='relu')
        self.do2 = Dropout(0.2)
        self.out = Dense(10, activation='softmax')
    
    def call(self, inputs):
        net = self.conv1(inputs)
        net = self.batch1(net)
        net = self.pool1(net)
        
        net = self.conv2(net)
        net = self.batch2(net)
        net = self.pool2(net)
        
        net = self.conv3(net)
        net = self.batch3(net)
        net = self.pool3(net)
        
        net = self.flat(net)
        
        net = self.d1(net)
        net = self.do1(net)
        net = self.d2(net)
        net = self.do2(net)
        return self.out(net)
    
    
model = MyModel()
temp_inputs = Input(shape=(28,28,1))
model(temp_inputs)

model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 6, 6, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 6, 6, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        36,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130,938 (511.48 KB)

 Trainable params: 130,490 (509.73 KB)

 Non-trainable params: 448 (1.75 KB)

In [6]:
# 모델 학습 방법 - 1
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss=loss_object, metrics=['acc'])

# 조기 종료
# es = EarlyStopping(patience=3, restore_best_weights=True)

# 학습
history = model.fit(x_train, y_train, epochs=10, batch_size=128, verbose=2) # 뒤에 3놈은 세트 max_queue_size=10, workers=4, use_multiprocessing=True

Epoch 1/10


2025-09-19 11:56:53.157330: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


469/469 - 12s - 26ms/step - acc: 0.8451 - loss: 0.5195
Epoch 2/10
469/469 - 10s - 22ms/step - acc: 0.9586 - loss: 0.1790
Epoch 3/10
469/469 - 10s - 21ms/step - acc: 0.9617 - loss: 0.1852
Epoch 4/10
469/469 - 10s - 22ms/step - acc: 0.9749 - loss: 0.1223
Epoch 5/10
469/469 - 11s - 24ms/step - acc: 0.9738 - loss: 0.1691
Epoch 6/10
469/469 - 10s - 22ms/step - acc: 0.9737 - loss: 0.2248
Epoch 7/10
469/469 - 10s - 21ms/step - acc: 0.9777 - loss: 0.2283
Epoch 8/10
469/469 - 10s - 20ms/step - acc: 0.9814 - loss: 0.2101
Epoch 9/10
469/469 - 10s - 21ms/step - acc: 0.9786 - loss: 0.2931
Epoch 10/10
469/469 - 10s - 21ms/step - acc: 0.9825 - loss: 0.3221


In [8]:
# 모델 평가 - 1. 가장 일반적인 방법
score = model.evaluate(x_test, y_test)
print(f'test loss : {score[0]:.4f}')
print(f'test acc : {score[1]:.4f}')

print(f'예측값 : {np.argmax(model.predict(x_test[:2]), 1)}')
print(f'실제값 : {y_test[:2]}')

# 모델 평가 - 2
train_loss, train_acc = model.evaluate(x_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'train_loss : {train_loss:.4f}, train_acc : {train_acc*100:.4f}%')
print(f'test_loss : {test_loss:.4f}, test_acc : {test_acc*100:.4f}%')

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - acc: 0.9627 - loss: 0.9802
test loss : 0.9802
test acc : 0.9627
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
예측값 : [7 2]
실제값 : [7 2]
train_loss : 0.9401, train_acc : 96.0850%
test_loss : 0.9802, test_acc : 96.2700%


In [15]:
# GradientTape 운영 - 모델 서브 프로세싱 학습방법
# 모델 손실과 성능을 측정할 자료 선택, 수집된 측정 지표를 바탕으로 최종 결과 출력을 위한 객체 생성
train_loss = Mean() # 주어진 값의 (가중) 평균을 계산
train_accuracy = SparseCategoricalAccuracy()

test_loss = Mean()
test_accuracy = SparseCategoricalAccuracy()

@tf.function    # autograph
def train_step(images, labels):     # 이 함수를 반복하면 loss를 최소화
    with tf.GradientTape() as tape:    # 진행 과정을 모두 저장
        # GradientTape : forward propagation이 진행되면서 나중에 backpropagation을 할 때 필요한 값들을 저장해둔다.
        
        predictions = model(images)
        loss = loss_object(labels, predictions)
        
    # 미분 계산
    gradients = tape.gradient(loss, model.trainable_variables)    # loss 최소화를 위한 미분 계산
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 5
for epoch in range(EPOCHS):
    for train_images, train_labels in train_ds:   # slice된 60,000개
        train_step(train_images, train_labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    template = 'epochs:{}\ntrain_loss:{:.4f}, train_acc:{:.4f}%\ntest_loss:{:.4f}, test_acc:{:.4f}%'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))
    
    

2025-09-19 12:24:26.914805: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-19 12:24:28.240599: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epochs:1
train_loss:0.6454, train_acc:98.4817%
test_loss:0.3782, test_acc:98.9000%


2025-09-19 12:24:52.211355: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-19 12:24:53.226325: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epochs:2
train_loss:0.5254, train_acc:98.6308%
test_loss:0.3435, test_acc:98.9350%


2025-09-19 12:25:16.479502: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-19 12:25:17.455737: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2025-09-19 12:25:41.190084: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-19 12:25:42.173815: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epochs:4
train_loss:0.4505, train_acc:98.6600%
test_loss:0.3443, test_acc:98.8575%


2025-09-19 12:26:05.384546: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epochs:5
train_loss:0.4927, train_acc:98.6130%
test_loss:0.4771, test_acc:98.6800%


2025-09-19 12:26:06.367266: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
